# Transformer Model Implementation 


In [1]:
# import required packages
import torch 
import torch.nn as nn 
import copy 
import math 
from torch.nn.functional import log_softmax
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
import spacy 


In [4]:
# Define a Token Embedding 
class TokenEmbeddings(nn.Embedding): 
    """
    Converting token into embedding vector
    """
    def __init__(self, vocab_size, d_model):
        """
        class for token embedding without positional encoding

        :param vocab_size: number of vocabs that TokenEmbeddings can handle
        :param d_model: dimension of embedding vector
        """
        super(TokenEmbeddings, self).__init__(vocab_size, d_model, padding_idx=1)

# Define Positional Encoding 
class PositionalEncoding(nn.Module): 
    """ 
    compute reusable sinusoid positional encoding
    """
    def __init__(self, d_model, max_len, device): 
        """
        construct sinusoid positional encoding that is going to be reused everytime when it is needed

        :param d_model: dimension of embedding vector
        :param max_len: maximum sequence length of token(a.k.a window size of attention method)
        """
        super(PositionalEncoding, self).__init__()

        # define a max_len * d_model size encoding matrix
        self.encoding = torch.zeros(max_len, d_model, device=device)

        # since positional encoding is not learnable, we turn off the gradient engine
        self.encoding.requires_grad = False

        # define a position at the sequence
        pos = torch.arange(0, max_len, device=device)
        # expand the max_len vector to max_len * 1 matrix 
        pos = pos.float().unsqueeze(dim=1)

        _2i = torch.arange(0, d_model, step=2, device=device).float()

        # define a sinusoid positional encoding
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))

    def forward(self, x): 
        batch_size, seq_len = x.shape

        return self.encoding[:seq_len, :]

# Define Transformer Embedding 
class TransformerEmbedding(nn.Module): 
    """
    token embedding + positional encoding
    """
    def __init__(self, vocab_size, d_model, max_len, drop_prob, device): 
        """
        initialize the embedding class for word+position embedding

        :param vocab_size: number of vocabs that TokenEmbeddings can handle
        :param d_model: dimension of embedding vector
        :param max_len: maximum sequence length of token(a.k.a window size of attention method)
        :param drop_prob: dropout probability to reduce overfitting
        """
        super(TransformerEmbedding, self).__init__()
        self.token_emb = TokenEmbeddings(vocab_size, d_model)
        self.position_emb = PositionalEncoding(d_model, max_len, device)
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x): 
        tok_emb = self.token_emb(x)
        pos_emb = self.position_emb(x)

        return self.dropout(tok_emb+pos_emb)


In [5]:
# Define Attention Block 
class AttentionBlock(nn.Module): 
    """
    compute scale dot product attention for Query, Key, Value
    """
    def __init__(self):
        super(AttentionBlock, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, eps=1e-12): 
        batch_size, head, length, d_tensor = k.shape

        # calculate the k_T
        k_T = k.transpose(2, 3)

        # calculate the attention weight 
        att_weight = (q @ k_T) / math.sqrt(d_tensor)

        # if there are any masks that needs to be applied
        if mask is not None:
            att_weight = att_weight.masked_fill(mask == 0, -10000)

        # calculate the softmax 
        # att_weight shape: batch_size, head, seq_len, seq_len
        att_weight = self.softmax(att_weight)

        # att_weight @ v shape: batch_size, head, seq_len, d_model
        return att_weight @ v, att_weight

# Define MultiHeadAttention Block 
class MultiHeadAttentionBlock(nn.Module): 
    """
    define multi head attention block using AttentionBlock module
    """
    def __init__(self, d_model, n_head): 
        """
        Multi-head self-attention utilize the parallelism of GPU

        :param d_model: dimension of embedding vector
        :param n_head: number of heads
        """
        super(MultiHeadAttentionBlock, self).__init__()
        self.n_head = n_head
        self.attention = AttentionBlock()
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)

        # in the paper, d_v = d_k = d_q
        self.Wv = nn.Linear(d_model, d_model)

        self.Wconcat = nn.Linear(d_model, d_model)

    def split(self, tensor): 
        """
        split the tensor by number of head 

        :param tensor: tensor of shape batch_size  * seq_len * d_model
        :return: return tensor of shape batch_size * n_head * seq_len * d_tensor
        """
        batch_size, seq_len, d_model = tensor.shape 

        d_tensor = d_model // self.n_head

        tensor = tensor.view(batch_size, seq_len, self.n_head, d_tensor).transpose(1, 2)

        return tensor 

    def concat(self, tensor): 
        """
        concat tensor. Inverse operation of split

        :param tensor: tensor of shape batch_size * n_head * seq_len * d_tensor 
        :return: return tensor of shape batch_size * seq_len * d_model
        """
        batch_size, n_head, seq_len, d_tensor = tensor.shape

        d_model = n_head * d_tensor
        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        return tensor 
    

    def forward(self, q, k, v, mask=None): 
        # apply linear transformation to derive q, k, v 
        q, k, v = self.Wq(q), self.Wk(k), self.Wv(v)

        # split the tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # apply attention to q, k, v 
        out, att_weight = self.attention(q, k, v, mask=mask)

        # concat 
        out = self.concat(out)

        # apply concat weight 
        out = self.Wconcat(out)
        return out 
        

In [6]:
# Define LayerNorm 
class LayerNorm(nn.Module): 
    """
    Normalize all features for each samples. 
    """
    def __init__(self, d_model, eps=1e-12): 
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps 

    def forward(self, x): 
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, keepdim=True, unbiased=False)

        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta

        return out 

In [7]:
# define FeedForward Network 
class FeedForwardBlock(nn.Module): 
    def __init__(self, d_model, hidden, drop_prob=0.1): 
        super(FeedForwardBlock, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden) 
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x): 
        x = self.linear1(x) 
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x) 
        return x 

In [8]:
# Define Encoder Layer 
class EncoderLayer(nn.Module): 
    def __init__(self, d_model, ffn_hidden, n_head, drop_prob): 
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttentionBlock(d_model, n_head)
        self.norm1 = LayerNorm(d_model) 
        self.dropout1 = nn.Dropout(drop_prob)

        self.ffn = FeedForwardBlock(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNorm(d_model)
        self.dropout2 = nn.Dropout(drop_prob)

    def forward(self, x, src_mask): 
        residual = x 
        x = self.attention(q=x, k=x, v=x, mask=src_mask)

        x = self.dropout1(x) 
        x = self.norm1(x + residual)

        residual = x 
        x = self.ffn(x) 

        x =  self.dropout2(x)
        x = self.norm2(x + residual)

        return x 

# Define Decoder Layer 
class DecoderLayer(nn.Module): 
    def __init__(self, d_model, ffn_hidden, n_head, drop_prob): 
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttentionBlock(d_model, n_head)
        self.norm1 = LayerNorm(d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.enc_dec_attention = MultiHeadAttentionBlock(d_model, n_head)
        self.norm2 = LayerNorm(d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = FeedForwardBlock(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNorm(d_model)
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, dec, enc, trg_mask, src_mask): 
        residual = dec
        x = self.self_attention(q=dec, k=dec, v=dec, mask=trg_mask)

        x = self.dropout1(x)
        x = self.norm1(x + residual)

        if enc is not None: 
            residual = x 
            x = self.enc_dec_attention(q=x, k=enc, v=enc, mask=src_mask)
            x = self.dropout2(x) 
            x = self.norm2(x + residual)

        residual = x 
        x = self.ffn(x)
        x = self.dropout3(x)
        x = self.norm3(x + residual)

        return x 

In [9]:
# Define Encoder Model
class Encoder(nn.Module): 
    """
    Encoder for Transformer
    """
    def __init__(self, enc_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device): 
        super(Encoder, self).__init__()
        self.emb = TransformerEmbedding(d_model=d_model, max_len=max_len, vocab_size=enc_voc_size, drop_prob=drop_prob, device=device)
        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model, ffn_hidden=ffn_hidden, n_head=n_head, drop_prob=drop_prob) for _ in range(n_layers)])

    def forward(self, x, src_mask): 
        x = self.emb(x) 
        for layer in self.layers: 
            x = layer(x, src_mask)

        return x

        
class Decoder(nn.Module): 
    """
    Decoder for Transformer
    """
    def __init__(self, dec_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device): 
        super(Decoder, self).__init__()
        self.emb = TransformerEmbedding(d_model=d_model, max_len=max_len, vocab_size=dec_voc_size, drop_prob=drop_prob, device=device)

        self.layers = nn.ModuleList([DecoderLayer(d_model=d_model, ffn_hidden=ffn_hidden, n_head=n_head, drop_prob=drop_prob) for _ in range(n_layers)])

        self.linear = nn.Linear(d_model, dec_voc_size)

    def forward(self, trg, enc_src, trg_mask, src_mask): 
        trg = self.emb(trg)
        for layer in self.layers: 
            trg = layer(trg, enc_src, trg_mask, src_mask) 

        output = self.linear(trg)
        return output
    

# Define Transformer Model 
class Transformer(nn.Module): 
    """
    Transformer Model
    """
    def __init__(self, src_pad_token, trg_pad_token, trg_sos_token, enc_voc_size, dec_voc_size, ffn_hidden, n_layers, drop_prob, device): 
        """
        Constructing Transformer Model 

        :param src_pad_token: embedding vector that represents <pad> in source 
        :param trg_pad_token: embedding vector that represents <pad> in target 
        :param trg_sos_token: embedding vector that represents <sos> in target 
        :params enc_voc_size: number of vocabs that encoderEmbedder can handle
        :params dec_voc_size: number of vocabs that decoderEmbedder can handle
        :params ffn_hidden: hidden vector dimension for fastfeedforward layer 
        :params n_layers: number of EncoderLayer/DecoderLayer used
        :params drop_prob: dropout probability
        """
        super(Transformer, self).__init__()

        self.src_pad_token = src_pad_token
        self.trg_pad_token = trg_pad_token
        self.trg_sos_token = trg_sos_token
        self.device = device
        self.encoder = Encoder(d_model=d_model, n_head=n_head, max_len=max_len, ffn_hidden=ffn_hidden, enc_voc_size=enc_voc_size, drop_prob=drop_prob, n_layers=n_layers, device=device)
        self.decoder = Decoder(d_model=d_model, n_head=n_head, max_len=max_len, ffn_hidden=ffn_hidden, dec_voc_size=dec_voc_size, drop_prob=drop_prob, n_layers=n_layers, device=device)

    def make_src_mask(self, src): 
        # print(f'src: {src}')
        # print(f'src_pad_token: {self.src_pad_token}')
        # print(f'src != self.src_pad_token: {src != self.src_pad_token}')
        src_mask = (src != self.src_pad_token).unsqueeze(1).unsqueeze(2)
        return src_mask

    def make_trg_mask(self, trg): 
        trg_pad_mask = (trg != self.trg_pad_token).unsqueeze(1).unsqueeze(3)
        trg_len = trg.shape[1]

        # make a look-ahead mask using torch.tril 
        # [[1 0 0]
        #  [1 1 0]
        #  [1 1 1]]
        trg_sub_mask = torch.tril(torch.ones(trg_len, trg_len)).type(torch.ByteTensor).to(self.device)

        trg_mask = trg_pad_mask & trg_sub_mask
        return trg_mask
        
    
    def forward(self, src, trg): 
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        output = self.decoder(trg, enc_src, trg_mask, src_mask)
        return output

Using MPS as device
